In [1]:
# workaround for incredibly stupid vscode on snap bug
from os import environ
krb5ccname = environ["KRB5CCNAME"]
if "hostfs" in krb5ccname:
    krb5ccname = krb5ccname.replace("/var/lib/snapd/hostfs", "")
    environ["KRB5CCNAME"] = krb5ccname

In [2]:
import ROOT
from analysis_framework import Dataset
from OptimalObservableHelper import OptimalObservableHelper
from AltSetupHandler import AltSetupHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8df0960


In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad
n_threads = 12
# prod = False
prod = True
no_rvec = True
write_outputs = False
# write_outputs = True
# plot_dir_postfix = "-new-cuts"
dataset_path = "data/datasets/reweighted/test.json"
# output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/reweighted/test"
# output_meta_path = "data/datasets/reweighted"
# output_meta = f"{output_meta_path}/test.json"
# checked_output_meta = f"{output_meta_path}/checked-test.json"
# output_collections = [
#     "true_lep_lvec", "true_nu_lvec", "true_quark1_lvec", "true_quark2_lvec",
#     "iso_lep_lvec", "nu_lvec", "R2Jet_sel1_lvec", "R2Jet_sel2_lvec",
#     ]
# true lvecs do not exist in every df so cannot be explicitly requested...
# urgh but empty snapshots are also not allowed
# output_collections = r"(true_\w+_lvec)|(iso_lep_lvec)|(nu_lvec)|(R2Jet_sel1_lvec)|(R2Jet_sel2_lvec)|(\w*sqme\w*)|(weight\w*)"
# plot_dir = f"plots/pre-selection/test{plot_dir_postfix}"
if prod:
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs-min-aa-min-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/min-higgs.json"
    dataset_path = "data/datasets/reweighted/signal-only.json"
    # output_path = "root://eospublic.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/min-higgs-d"
    # output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/reweighted/signal-only"
    # output_meta_path = "data/datasets/reweighted"
    # output_meta = f"{output_meta_path}/signal-only.json"
    # checked_output_meta = f"{output_meta_path}/checked-signal-only.json"
    # plot_dir = "plots/pre-selection/full"
    # plot_dir = f"plots/pre-selection/min-higgs{plot_dir_postfix}"


In [4]:
ROOT.EnableImplicitMT(n_threads)
# environ["OMP_NUM_THREADS"] = "6"

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = OptimalObservableHelper(dataset)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xd0bb8a0


In [7]:
analysis.init_categories()
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]

True


In [8]:
oo_configs = [
    "g1z_pos_1em08",
    "ka_pos_1em08",
    "la_pos_1em08",
    ]
analysis.define_optimal_observables_truth(oo_configs, truth_categories=signal_category)
analysis.define_optimal_observables_truth_averaged(oo_configs, truth_categories=signal_category)
analysis.define_optimal_observables_reco(oo_configs)
oo_names = [f"O_{c}" for c in oo_configs]

In [9]:
analysis.add_filter("&&".join([f"std::isfinite({oo})" for oo in oo_names]), "finite OO")
analysis.add_filter("&&".join([f"abs({oo}) <= 5" for oo in oo_names]), "abs(OO) <= 5")
analysis.book_reports()

In [10]:
analysis.book_histogram_1D("mc_O_g1z_pos_1em08", "mc_O_g1z_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("mc_O_ka_pos_1em08", "mc_O_ka_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("mc_O_la_pos_1em08", "mc_O_la_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
# analysis.book_histogram_1D("mc_O_g1z_pos_1em08", "mc_O_g1z_pos_1em08", ("", "", 250, -5, 5), categories=signal_category)
# analysis.book_histogram_1D("mc_O_ka_pos_1em08", "mc_O_ka_pos_1em08", ("", "", 250, -5, 5), categories=signal_category)
# analysis.book_histogram_1D("mc_O_la_pos_1em08", "mc_O_la_pos_1em08", ("", "", 250, -5, 5), categories=signal_category)

analysis.book_histogram_1D("O_g1z_pos_1em08", "O_g1z_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("O_ka_pos_1em08", "O_ka_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("O_la_pos_1em08", "O_la_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
# analysis.book_histogram_1D("O_g1z_pos_1em08", "O_g1z_pos_1em08", ("", "", 250, -5, 5), categories=signal_category)
# analysis.book_histogram_1D("O_ka_pos_1em08", "O_ka_pos_1em08", ("", "", 250, -5, 5), categories=signal_category)
# analysis.book_histogram_1D("O_la_pos_1em08", "O_la_pos_1em08", ("", "", 250, -5, 5), categories=signal_category)
analysis.book_histogram_1D("av_mc_O_g1z_pos_1em08", "av_mc_O_g1z_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("av_mc_O_ka_pos_1em08", "av_mc_O_ka_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("av_mc_O_la_pos_1em08", "av_mc_O_la_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)

In [11]:
alt_setup_handler = AltSetupHandler("""
{
  "SM": {
    "g1z": 1.0,
    "ka": 1.0,
    "la": 0.0
  },
"variations": [
    0.001,
    -0.001,
    1e-04,
    -1e-04,
    1e-08
  ]
}
""", mirror=False, combinations=False)
alt_config_names = list(alt_setup_handler.get_alt_setup().keys())

In [12]:
mc_oo_names = [f"mc_{oo}" for oo in oo_names]
av_mc_oo_names = [f"av_mc_{oo}" for oo in oo_names]

In [13]:
weight_names = analysis.book_weight_sums(["nominal"] + alt_config_names, categories=signal_category)

In [14]:
analysis.define_weighted_oo(oo_names, weight_names, categories=signal_category)
analysis.define_weighted_oo(mc_oo_names, weight_names, categories=signal_category)
analysis.define_weighted_oo(av_mc_oo_names, weight_names, categories=signal_category)

In [15]:
analysis.book_oo_sums(oo_names, weight_names, categories=signal_category)
analysis.book_oo_sums(mc_oo_names, weight_names, categories=signal_category)
analysis.book_oo_sums(av_mc_oo_names, weight_names, categories=signal_category)

In [16]:
analysis.book_oo_matrix(oo_names, categories=signal_category)
analysis.book_oo_matrix(mc_oo_names, categories=signal_category)
analysis.book_oo_matrix(av_mc_oo_names, categories=signal_category)

In [17]:
if write_outputs:
    analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [18]:
%%time
analysis.run()

CPU times: user 13.7 s, sys: 783 ms, total: 14.5 s
Wall time: 5.05 s


In [19]:
analysis.print_reports()
# if write_outputs:
    # analysis.check_snapshots("events", output_path, checked_output_meta)

         4f_sw_sl_signal               4f_sl_bkg
        10244400 (1e+04)           33239 (6e+02) All
        10236856 (1e+04)           33227 (6e+02) finite OO
        10111886 (1e+04)           32976 (6e+02) abs(OO) <= 5
                    0.99                    0.99 efficiency



In [20]:
# analysis.draw_histogram("mc_O_g1z_pos_1em08", categories=signal_category)
# analysis.draw_histogram("av_mc_O_g1z_pos_1em08", categories=signal_category)
# analysis.draw_histogram("O_g1z_pos_1em08", categories=signal_category)

# analysis.draw_histogram("mc_O_ka_pos_1em08", categories=signal_category)
# analysis.draw_histogram("av_mc_O_ka_pos_1em08", categories=signal_category)
# analysis.draw_histogram("O_ka_pos_1em08", categories=signal_category)

# analysis.draw_histogram("mc_O_la_pos_1em08", categories=signal_category)
# analysis.draw_histogram("av_mc_O_la_pos_1em08", categories=signal_category)
# analysis.draw_histogram("O_la_pos_1em08", categories=signal_category)

In [21]:
# analysis.compare_histograms_unscaled(["av_mc_O_g1z_pos_1em08", "mc_O_g1z_pos_1em08", "O_g1z_pos_1em08"], categories=signal_category)
# analysis.compare_histograms_unscaled(["av_mc_O_ka_pos_1em08", "mc_O_ka_pos_1em08", "O_ka_pos_1em08"], categories=signal_category)
# analysis.compare_histograms_unscaled(["av_mc_O_la_pos_1em08", "mc_O_la_pos_1em08", "O_la_pos_1em08"], categories=signal_category)

In [22]:
# calculate means etc.
oo_means = analysis.calc_oo_means(oo_names, weight_names, categories=signal_category)
mc_oo_means = analysis.calc_oo_means(mc_oo_names, weight_names, categories=signal_category)
av_mc_oo_means = analysis.calc_oo_means(av_mc_oo_names, weight_names, categories=signal_category)

In [23]:
print(alt_config_names)

['g1z_pos_1em03', 'ka_pos_1em03', 'la_pos_1em03', 'g1z_neg_1em03', 'ka_neg_1em03', 'la_neg_1em03', 'g1z_pos_1em04', 'ka_pos_1em04', 'la_pos_1em04', 'g1z_neg_1em04', 'ka_neg_1em04', 'la_neg_1em04', 'g1z_pos_1em08', 'ka_pos_1em08', 'la_pos_1em08']


In [24]:
oo_graphs = analysis.make_slope_graphs(oo_names, ["g1z", "ka", "la"], alt_config_names, oo_means)

In [25]:
canvases = {}
for k, g in oo_graphs.items():
    c = ROOT.TCanvas()
    g.Draw("alp")
    c.Draw()
    canvases[k] = c

In [26]:
oo_slopes = analysis.get_slopes(oo_names, ["g1z", "ka", "la"], oo_means)
print(oo_slopes)
mc_oo_slopes = analysis.get_slopes(mc_oo_names, ["g1z", "ka", "la"], mc_oo_means)
print(mc_oo_slopes)
av_mc_oo_slopes = analysis.get_slopes(av_mc_oo_names, ["g1z", "ka", "la"], av_mc_oo_means)
print(av_mc_oo_slopes)

{'O_g1z_pos_1em08_g1z': 16.453997746930632, 'O_g1z_pos_1em08_ka': 3.6333378331753816, 'O_g1z_pos_1em08_la': -9.742560084453636, 'O_ka_pos_1em08_g1z': 1.011958347460184, 'O_ka_pos_1em08_ka': 3.7941217592848138, 'O_ka_pos_1em08_la': 0.008583486433210117, 'O_la_pos_1em08_g1z': -5.051637323324201, 'O_la_pos_1em08_ka': -0.6220281135795896, 'O_la_pos_1em08_la': 9.088317890899981}
{'mc_O_g1z_pos_1em08_g1z': -14.656818651914932, 'mc_O_g1z_pos_1em08_ka': 0.13799156786774744, 'mc_O_g1z_pos_1em08_la': 8.216081994872162, 'mc_O_ka_pos_1em08_g1z': 0.22007671557996403, 'mc_O_ka_pos_1em08_ka': -6.9871550530120485, 'mc_O_ka_pos_1em08_la': 1.05630555511775, 'mc_O_la_pos_1em08_g1z': -10.111059819929482, 'mc_O_la_pos_1em08_ka': -0.8150795731844511, 'mc_O_la_pos_1em08_la': 29.224092847288315}
{'av_mc_O_g1z_pos_1em08_g1z': -7.710755865670277, 'av_mc_O_g1z_pos_1em08_ka': -1.5784309677053208, 'av_mc_O_g1z_pos_1em08_la': 4.74644754663656, 'av_mc_O_ka_pos_1em08_g1z': -2.5092279664723973, 'av_mc_O_ka_pos_1em08_k

In [27]:
analysis.print_fit_input(oo_names, weight_names, e_pol=0., p_pol=0., categories=signal_category, dir="fit-configs/signal-only", name="reco_oo")
analysis.print_fit_input(mc_oo_names, weight_names, e_pol=0., p_pol=0., categories=signal_category, dir="fit-configs/signal-only", name="mc_oo")
analysis.print_fit_input(av_mc_oo_names, weight_names, e_pol=0., p_pol=0., categories=signal_category, dir="fit-configs/signal-only", name="av_mc_oo")

#evt/ab_inv, means, slopes, cov
2022377.202575377
[0.025102206287087838, 0.04795018914790096, 0.0647069553362756]
[[16.453997746930632, 3.6333378331753816, -9.742560084453636], [1.011958347460184, 3.7941217592848138, 0.008583486433210117], [-5.051637323324201, -0.6220281135795896, 9.088317890899981]]
[[0.526004905526162, 0.11740744424050836, -0.2590917059455628], [0.11740744424050836, 0.3730411566425547, 0.024186149615180685], [-0.2590917059455628, 0.024186149615180685, 1.033176508820112]]

#evt/ab_inv, means, slopes, cov
2022377.202575377
[-0.09139651922247422, -0.05730706691798843, 0.07426731796303587]
[[-14.656818651914932, 0.13799156786774744, 8.216081994872162], [0.22007671557996403, -6.9871550530120485, 1.05630555511775], [-10.111059819929482, -0.8150795731844511, 29.224092847288315]]
[[1.3395822044707593, -0.012611952435148288, -0.7509212939840509], [-0.012611952435148288, 0.40041336031283115, -0.06053377393436329], [-0.7509212939840509, -0.06053377393436329, 2.1703949960754483]

In [28]:
mc_M = analysis.get_oo_matrix([f"mc_{oo}" for oo in oo_names], categories=signal_category)
av_mc_M = analysis.get_oo_matrix([f"av_mc_{oo}" for oo in oo_names], categories=signal_category)

In [29]:
import numpy as np
from itertools import combinations_with_replacement
mc_C_tilde = np.zeros((3,3))
av_mc_C_tilde = np.zeros((3,3))
for k, (i, j) in enumerate(combinations_with_replacement(range(3), 2)):
    mc_C_tilde[i, j] = mc_M[k]
    mc_C_tilde[j, i] = mc_M[k]
    av_mc_C_tilde[i, j] = av_mc_M[k]
    av_mc_C_tilde[j, i] = av_mc_M[k]
print(mc_C_tilde)
print(av_mc_C_tilde)

[[13630170.41383247   -74567.93937741 -7661867.72995559]
 [  -74567.93937741  4082142.70154957  -655147.23514477]
 [-7661867.72995559  -655147.23514477 22002560.27076217]]
[[ 7210538.3575505   1499890.32417239 -4464806.2934658 ]
 [ 1499890.32417239  2640440.32547725  -830845.06320645]
 [-4464806.2934658   -830845.06320645  8540660.33992035]]


In [30]:
print(np.sqrt(np.diag(np.linalg.inv(mc_C_tilde))))
print(np.sqrt(np.diag(np.linalg.inv(av_mc_C_tilde))))

[0.00030234 0.00049664 0.00023852]
[0.00047496 0.00065557 0.00041624]


In [31]:
n_events = analysis.get_sum("weight_nominal", categories=signal_category)

mc_oo_means_vec = np.asarray([mc_oo_means[f"mc_{oo}_weight_nominal"] for oo in oo_names])
av_mc_oo_means_vec = np.asarray([av_mc_oo_means[f"av_mc_{oo}_weight_nominal"] for oo in oo_names])
mc_C = mc_C_tilde / n_events - np.outer(mc_oo_means_vec, mc_oo_means_vec)
av_mc_C = av_mc_C_tilde / n_events - np.outer(av_mc_oo_means_vec, av_mc_oo_means_vec)
print(mc_C)
print(av_mc_C)

[[ 1.3395822  -0.01261195 -0.75092129]
 [-0.01261195  0.40041336 -0.06053377]
 [-0.75092129 -0.06053377  2.170395  ]]
[[ 0.70474081  0.14308536 -0.43483305]
 [ 0.14308536  0.25782294 -0.07794502]
 [-0.43483305 -0.07794502  0.83921821]]


In [32]:
print(mc_oo_means_vec)
print(av_mc_oo_means_vec)

[-0.09139652 -0.05730707  0.07426732]
[-0.09129463 -0.05744124  0.0734694 ]


In [33]:
# precision result is very very close to Figure 13 in https://arxiv.org/pdf/1907.04311 :)
# which is weird as we are missing all the muons here so should be factor sqrt(2) worse
# instead we are factor sqrt(2), and a tiny bit, better than standalone whizard without selection cuts
# and the correlations are completely off as well
print(np.linalg.inv(mc_C))
V_hg = np.linalg.inv(mc_C) / n_events
precision = np.sqrt(np.diag(V_hg))
print(precision)
# but correlations are different than
# https://github.com/silvest/HEPfit/blob/master/Configurations/Publications/arXiv_1907.04311v1/config_CEPC/aTGC_observables_CEPC_OO.conf
# https://github.com/silvest/HEPfit/blob/master/Configurations/Publications/arXiv_1907.04311v1/config_FCCee/aTGC_observables_FCCee_OO.conf
# which themselves are also different?? I guess they include also the multiple runs already and that is their difference??
correlation = V_hg / np.outer(precision, precision)
print(correlation)

[[0.92854673 0.07814405 0.32344153]
 [0.07814405 2.51457042 0.09716963]
 [0.32344153 0.09716963 0.57536125]]
[0.00030303 0.00049867 0.00023854]
[[1.         0.05114017 0.44251051]
 [0.05114017 1.         0.08078455]
 [0.44251051 0.08078455 1.        ]]
